# Question Answering Fine-tuning 

La siguiente tarea consiste en entrenar un modelo de HuggingFace (HF) para realizar la _task_ de _question_answering_. El dataset para entrenar dicho modelo está predefinido. Sin embargo, el modelo, el tokenizador y el trainer pueden ser totalmente personalizados. Es decir, que tendréis que realizar un trabajo de investigación, de prueba y error, para poder ir aprendiendo y ganando destreza con HF. 

Recomendaciones: 
- Durante este proceso, tendréis muchas dudas y encontraréis muchos errores. Tratad de resolverlas primero por vuestra cuenta, enteniendo la causa del error. Después con recursos online. Y, finalmente, siempre está el foro, que puede ser utilizado de forma participativa.
- No dejeis la tarea para el último día. Los modelos tardan en entrenar. Los problemas no se resuelven en la primera iteración.

Finalmente, se pide:
- Limpieza rigurosa en la presentación del notebook.
- El notebook se entrega con todas las celdas ejecutadas.
- Los comentarios (opcionales), mejor sobre el código con '#'. 

Ánimo!

## Dataset

A continuación, descargarás un dataset llamado _squad_ que contiene 87599 filas en el dataset de train y 10570 registros en el dataset de validation. 

Lo primero que tendrás que hacer es construir un dataset nuevo, llamado **ds_tarea**, que filtre el anterior dataset para quedarse con los registros que tengan el contenido de la columna _context_  con menos (estrictamente) de 300 caracteres.

In [18]:
import collections
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import os

# Cargar el dataset de SQuAD
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
dataset = load_dataset("squad")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [19]:
def filter_short_contexts(example):
    return len(example['context']) < 300
ds_tarea = dataset.filter(filter_short_contexts)


In [20]:
assert len(ds_tarea['train']) == 3466
assert len(ds_tarea['validation']) == 345

## EDA

Si tenéis que realizar alguna exploración del datos, utilizad esta sección.

In [21]:
# Celdas de libre uso

## Feature Engineering

Si tenéis que realizar alguna modificación de los datos (no siempre es necesaria, pero algunos modelos preentrenados lo piden), podéis utilizar esta sección. 

Al finalizar la sección, bien si modificais el dataset, bien si no lo modificáis, lo guardaréis en un dataset llamado __ds_tarea_featured__.

In [22]:
# Celdas de libre uso

In [23]:
ds_tarea_featured = ds_tarea # Esta línea tiene sentido en caso de que no se modifique el dataset

In [24]:
assert len(ds_tarea_featured['train']) == 3466
assert len(ds_tarea_featured['validation']) == 345

## Model and Tokenizer

El modelo finalmente escogido para hacer el fine-tuning, declaradlo en la variable _model_checkpoint_. Con dicho modelo seleccionado, se pide guardar el modelo y el tokenizador en las variables _model_ y _tokenizer_.

In [25]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_positions.append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_positions.append(token_end_index + 1)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples

tokenized_datasets = ds_tarea.map(prepare_train_features, batched=True, remove_columns=ds_tarea["train"].column_names)

ds_tarea_featured = tokenized_datasets
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

ds_tarea_featured = tokenized_datasets

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Fine-tuning

A continuación, de forma libre se pide entrenar un modelo de HuggingFace deseado. Se pide usar un Trainer de HuggingFace que tenga los siguientes argumentos como mínimo (puede haber más argumentos en todas las variables):

In [26]:
from datasets import load_metric

def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [27]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tarea_featured["train"],
    eval_dataset=ds_tarea_featured["validation"],
    tokenizer=tokenizer,
)

A continuación se entrena el modelo

In [28]:
trainer.train()























                                       
                                              

  0%|          | 0/651 [27:50<?, ?it/s]        



{'eval_loss': 2.346835136413574, 'eval_runtime': 22.803, 'eval_samples_per_second': 15.13, 'eval_steps_per_second': 0.965, 'epoch': 1.0}
























                                       
                                              

  0%|          | 0/651 [50:54<?, ?it/s]        



{'eval_loss': 2.1621029376983643, 'eval_runtime': 18.5088, 'eval_samples_per_second': 18.64, 'eval_steps_per_second': 1.189, 'epoch': 2.0}


                                       
  0%|          | 0/651 [57:22<?, ?it/s]          

{'loss': 2.3041, 'grad_norm': 17.857250213623047, 'learning_rate': 4.639016897081414e-06, 'epoch': 2.3}
























                                       
                                                

  0%|          | 0/651 [1:12:27<?, ?it/s]      

                                         
100%|██████████| 651/651 [1:10:36<00:00,  6.51s/it]

{'eval_loss': 2.171020030975342, 'eval_runtime': 18.5466, 'eval_samples_per_second': 18.602, 'eval_steps_per_second': 1.186, 'epoch': 3.0}
{'train_runtime': 4236.1826, 'train_samples_per_second': 2.455, 'train_steps_per_second': 0.154, 'train_loss': 2.11793944656208, 'epoch': 3.0}


TrainOutput(global_step=651, training_loss=2.11793944656208, metrics={'train_runtime': 4236.1826, 'train_samples_per_second': 2.455, 'train_steps_per_second': 0.154, 'total_flos': 1018898200599552.0, 'train_loss': 2.11793944656208, 'epoch': 3.0})

In [29]:
# trainer.save_model("xxx") Este código por si queréis salvarlo

## Evaluation

En este apartado, no vamos a entrar esta vez métrics. Lo que se va a pedir, es tomar dos ejemplos del dataset de evaluación. 

Con ambos ejemplos, vamos a ver cómo responden a las preguntas.

In [30]:
sample1 = 55
sample2 = 159

In [31]:
context1 = ds_tarea['validation'][sample1]['context']
question1 = ds_tarea['validation'][sample1]['question']
answer1 = ds_tarea['validation'][sample1]['answers']

ds_tarea['validation'][sample1]

{'id': '56bec0353aeaaa14008c934f',
 'title': 'Super_Bowl_50',
 'context': 'In addition to the Vince Lombardi Trophy that all Super Bowl champions receive, the winner of Super Bowl 50 will also receive a large, 18-karat gold-plated "50". Each digit will weigh 33 lb (15 kg) for a total of 66 lb (30 kg). Like the Lombardi Trophy, the "50" will be designed by Tiffany & Co.',
 'question': 'What will the "50" given to the Super Bowl winner weigh in pounds?',
 'answers': {'text': ['66', '66 lb', '33'], 'answer_start': [213, 213, 184]}}

In [32]:
context2 = ds_tarea['validation'][sample2]['context']
question2 = ds_tarea['validation'][sample2]['question']
answer2 = ds_tarea['validation'][sample2]['answers']

ds_tarea['validation'][sample2]

{'id': '56e11c24e3433e1400422c18',
 'title': 'Nikola_Tesla',
 'context': 'Tesla was 6 feet 2 inches (1.88 m) tall and weighed 142 pounds (64 kg), with almost no weight variance from 1888 to about 1926.:292 He was an elegant, stylish figure in New York City, meticulous in his grooming, clothing, and regimented in his daily activities.',
 'question': 'How much did Tesla weigh?',
 'answers': {'text': ['142 pounds', '142 pounds', '142 pounds (64 kg)'],
  'answer_start': [52, 52, 52]}}

Aquí se pide hacer la inferencia del modelo entrenado y poner los resultados en las variables _response1_ y _response2_.

In [43]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

response1 = qa_pipeline(question=question1, context=context1)
response2 = qa_pipeline(question=question2, context=context2)

In [44]:
response1
#responde de forma errada porque se confunde con otro de los datos que se encuentra en el contexto, pero de igual forma le da un bajo score

{'score': 0.048418860882520676,
 'start': 184,
 'end': 196,
 'answer': '33 lb (15 kg'}

In [37]:
response2
#responde de forma correcta

{'score': 0.2589314579963684,
 'start': 52,
 'end': 69,
 'answer': '142 pounds (64 kg'}